In [81]:
# === Header ===
#
# @date: 03 / 07 / 2019 (Xi'an)
# @author: ZHE LI
# @title: Learned KD-Tree
#
import math
import statistics
import numpy as np
from numpy import genfromtxt

In [56]:
# === Generate Query ===
#
# by default, use Gaussian distribution to generate the synthetic query workload
#
# @center_amount: assume the queries are clustered into that amount of clusters; integer
# @point_amount: the number of queries in each cluster; interger
# @dimensions: the domain of each dimension, the range should at least 8 times the sigma value; array object
# @sigma: the sigma of Gaussian distribution of each dimension; array object
#
# return @temp_queries: array object
#
import random
def generate_query(center_amount=10, point_amount=10, dimensions=[[-180,180],[-90,90]], sigma=[10,5]):
    
    random.seed()
    num_dims = len(dimensions)
    
    queries = []
    for j in range(2*num_dims):
        queries.append([])
    
    # for each cluster
    for i in range(center_amount):
        
        # determin the cluster center on each dimension
        cluster_center = []
        
        # for each dimension
        for j in range(num_dims):
            
            # to assure the distribution do not exceed the border
            dim_range = dimensions[j][1]-dimensions[j][0] - 8*sigma[j]
            center1D = random.randrange(dim_range)
            center1D += (dimensions[j][0] + 4*sigma[j])
            cluster_center.append(center1D)
            
            # for each query
            for k in range(point_amount):
                
                #determin the lower bound
                lower1D = random.gauss(center1D, sigma[j])
                upper1D = random.gauss(center1D, sigma[j])
                
                if lower1D > upper1D:
                    lower1D, upper1D = upper1D, lower1D
                    
                queries[2*j].append(lower1D)
                queries[2*j+1].append(upper1D)
                
    # transform the query structure
    temp_queries = []
    values = []
    for i in range(len(queries[0])):
        for j in range(2*num_dims):
            values.append(queries[j][i])
        temp_queries.append(values)
        values=[]
            
    # return queries
    return temp_queries

In [57]:
# Example of usage (Generate Query)
#
dimensions_ = [[1,1.20000000e+07],[1,4.00000000e+05],[1,2.00000000e+04]]
query_collection = generate_query(center_amount=1000, point_amount=10, dimensions=dimensions_, sigma=[50000,500,50])

In [58]:
# === Query Bounding ===
#
# this works for one dimension only !!! An implementation of query bounding.
#
# @query_collection: the queries in one single dimension; numpy object
#
# return @bounded_intervals: array object
#
def getoverlap(al, au, bl, bu):
    return max(0, min(au,bu)-max(al,bl))

def bounding_union(query_collection):
    
    # should keep it ordered first by the lower interval !!!!!!
    query_collection = query_collection[query_collection[:,0].argsort()]
    
    remaining_query = query_collection
    bounded_intervals = []
    
    while len(remaining_query) != 0:
        
        initial_interval = [remaining_query[0][0], remaining_query[0][1]]
        temp_interval = []
        
        for i in range(len(remaining_query)):
            
            overlap = getoverlap(initial_interval[0],initial_interval[1],remaining_query[i][0], remaining_query[i][1])
            
            # there is no overlap
            if overlap == 0:
                temp_interval.append([remaining_query[i][0], remaining_query[i][1]])
            else: # update interval border
                initial_interval[0] = min(initial_interval[0], remaining_query[i][0])
                initial_interval[1] = max(initial_interval[1], remaining_query[i][1])
                
        bounded_intervals.append(initial_interval)
        remaining_query = temp_interval
    
    return bounded_intervals

In [59]:
# Example of usage (Query Bounding)
#
def bound_queries(query_collection):
    # number of dimensions
    dims = int(len(query_collection[0])/2)
    bounded_queries = []
    for i in range(dims):
        queries_1dim = query_collection[:,2*i:2*i+2]
        bounded_intervals = bounding_union(queries_1dim)
        bounded_queries.append(bounded_intervals)
        #np.savetxt('/Users/lizhe/Desktop/LearnedKDTree/DataAndWorkload/SyntheticWorkload/Dim'+str(i)+'_QueryBound_TPCH_C1000_P10_S100.csv',bounded_intervals,delimiter=',')
    return bounded_queries

np.random.shuffle(query_collection)
training_set_percentage = 0.9
training_set_size = int(training_set_percentage * len(query_collection))
query_collection = np.asarray(query_collection) # as the shuffle make np to array, we need to make it back

training_set = query_collection[0:training_set_size,:]
test_set = query_collection[training_set_size:-1,:]

bounded_queries = bound_queries(training_set)

In [65]:
#np.savetxt('/Users/lizhe/Desktop/LearnedKDTree/DataAndWorkload/SyntheticWorkload/SyntheticWorkload_TestSet_TPCH_C1000_P10_S50000_500_50.csv',test_set,delimiter=',')

In [60]:
# === Learned KD-Tree Split ===
#
# asssumption: the query boundings will not overlap. divide the KD-Tree recursively
#
# @dataset: contains the data only in this subnode; numpy object, place it in the order of Dimorder
# @query_bound: contains all the bounds(i.e., bounded_queries), place it in the order of Dimorder; numpy object
# @currentDim: the dimension this iteration should focus on, an index in the Dimorder; integer
# @Dimorder: the order to split the dimensions; array object
# @domains: the current domain of the node of every dimension [first lower, second upper],[]...; array object
# @threshold: maximum page size
# @level: the current tree depth
#
# return @kdnodes: contains the domain of each node and the correpsonding records amount, notice the domain is
# ordered by Dimorder, the same as order of domains.
#
def ResuriveDivide(dataset, query_bound, currentDim, Dimorder, domains, threshold, level):
    
    #print("level: ",level)
    #print("dataset size: ",len(dataset))
    
    # check if the threshold is already satisfied
    total_size = len(dataset)
    if total_size <= threshold:
        # the kdnodes should be an global object outside the function
        kdnodes = []
        kdnodes.append([domains,total_size])
        return kdnodes
    
    # the current dimension
    # divideDim = Dimorder[currentDim]
    divideDim = currentDim # the dataset is already ordered as the Dimorder
    
    # sort according to the current dimension
    dataset = dataset[dataset[:,divideDim].argsort()]
    
    # find the medium
    medium = dataset[int(total_size/2),divideDim]
    medium_low = domains[divideDim][0]
    medium_up = domains[divideDim][1]
    
    # start check split position from the medium
    split_position = int(total_size/2)
    split_low = 0
    split_up = total_size
    
    # check if the split position intersect some query boundings in this dim
    for i in range(len(query_bound[divideDim])):
        
        # if intersect some query bounds
        if medium > query_bound[divideDim][i][0] and medium < query_bound[divideDim][i][1]:
            
            # check if the two end already exceeds domain
            if query_bound[divideDim][i][0] < domains[divideDim][0] and query_bound[divideDim][i][1] > domains[divideDim][1]:
                break;
            
            else:
                if query_bound[divideDim][i][0] > domains[divideDim][0]:
                # get the number of records from medium to the end
                    for j in range(split_position-1,-1,-1):
                        if dataset[j][divideDim] <= query_bound[divideDim][i][0]:
                            split_low = j
                            medium_low = dataset[split_low,divideDim]
                            break
                
                if query_bound[divideDim][i][1] < domains[divideDim][1]:
                # get the number of records from medium to the end
                    for j in range(split_position,total_size,1):
                        if dataset[j][divideDim] >= query_bound[divideDim][i][1]:
                            split_up = j
                            medium_up = dataset[split_up,divideDim]
                            break
                
            # if not exceeds then choose the one that is closest from the medium (in terms of #records!)
            if (total_size/2) - split_low < (split_up - total_size/2) and split_low != 0:
                split_position = split_low
                medium = medium_low
            elif (total_size/2) - split_low >= (split_up - total_size/2) and split_up != total_size :
                split_position = split_up
                medium = medium_up
            
            # after handle the overlap bounding, we can skip the remaining, as there will be at most 1 as assumned
            break;
            
    # split the dataset according to the split position
    sub_dataset1 = dataset[0:split_position,:]
    sub_dataset2 = dataset[split_position:-1,:]
    
    # change the domains
    sub_domains1 = np.copy(domains)
    sub_domains1[divideDim][1] = medium
    sub_domains2 = np.copy(domains)
    sub_domains2[divideDim][0] = medium
    
    # change the divideDim
    currentDim += 1
    if currentDim >= len(Dimorder):
        currentDim %= len(Dimorder)
    
    # used to see the current depth
    level += 1
    
    # recursion
    kdnodes = []
    kdnodes.extend(ResuriveDivide(sub_dataset1, query_bound, currentDim, Dimorder, sub_domains1, threshold, level))
    kdnodes.extend(ResuriveDivide(sub_dataset2, query_bound, currentDim, Dimorder, sub_domains2, threshold, level))
    
    # print("kdnodes: ",len(kdnodes))
    
    return kdnodes

In [61]:
# Example of usage (Learned KD-Tree Split)
#
Dimorder_ = [0,1,2] # should be the dimensions that is already bounded before
domains_ = [[1,1.20000000e+07],[1,4.00000000e+05],[1,2.00000000e+04]] # correspond to the above order
dataset = genfromtxt('/Users/lizhe/Desktop/LearnedKDTree/DataAndWorkload/SyntheticData/TPCH_12M_8Field.csv', delimiter=',')
dataset = dataset[:,Dimorder_] # retrieve only the used dimension, auto ordered into the Dimorder way
kdnodes = ResuriveDivide(dataset, bounded_queries, 0, Dimorder_, domains_, 32000, 0)
print(len(kdnodes))

558


In [62]:
# === Performance Evaluation ===
#
# evaluate the blocks of data to be fetched, when physical data of kdnodes are seperate !!!
# @queries: a collection of queries contains the lower and upper value in all dimensions; numpy object
# @kdnodes: the kdnodes generated above; array object
#
def Query(queries, kdnodes):
    
    counts = []
    count_single_query = 0;
    
    # number of dimensions
    dims = int(len(queries[0])/2)
    
    # for each query
    for i in range(len(queries)):
        
        count_single_query = 0
        
        # check for intersection for each kdnode
        for j in range(len(kdnodes)):
            
            # for each dimension
            intersection_tag = True
            for k in range(dims):
                
                # an intersection holds if it intersecs in all dimensions
                if queries[i][2*k] >= kdnodes[j][0][k][1] or queries[i][2*k+1] <= kdnodes[j][0][k][0]:
                    intersection_tag = False
                    break
                
            # if the query intersect with this kdnode
            if intersection_tag:
                count_single_query += 1
            
        counts.append(count_single_query)
    
    print("blocks IO: ", counts)
    print("blocks IO(average): ", statistics.mean(counts))

In [63]:
# Example of usage (Performance Evaluation)
#
Query(test_set, kdnodes) # the queries generated in Query Bounding use 10% of the generated queries, should be in Dimorder order

blocks IO:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [79]:
# === Physical Storage ===
#
# Reform the kdnodes from seperate physcial stroage to dense, continous physical storage
#
# @kdnodes: the previous generated kdnodes, already in order; array
# @threshold: the maximum page size 
#
# return @dense_kdnodes: array
#
def DenseKDNodes(kdnodes, threshold):
    
    dense_kdnodes = []
    previous_records = 0
    current_records = 0
    page_count = 0
    
    for i in range(len(kdnodes)):
        previous_records = current_records
        current_records += kdnodes[i][1]
        if current_records > threshold:
            # determine how many pages exceeds
            remaining = kdnodes[i][1] - (threshold - previous_records)
            num_pages = math.ceil(remaining / threshold) # num of new pages required
            pages = [i+page_count for i in range(num_pages+1)]
            page_count += num_pages
            current_records = remaining % threshold
            dense_kdn = [kdnodes[i][0],pages]
            dense_kdnodes.append(dense_kdn)
        else:
            dense_kdn = [kdnodes[i][0],[page_count]]
            dense_kdnodes.append(dense_kdn)
                         
    return dense_kdnodes

In [82]:
# Example of usage (Physical Storage)
#
dense_kdnodes = DenseKDNodes(kdnodes, 32000)

In [93]:
print(dense_kdnodes)

[[array([[  1.00000000e+00,   9.89830000e+04],
       [  1.00000000e+00,   9.27000000e+04],
       [  1.00000000e+00,   1.98870000e+04]]), [0]], [array([[  1.00000000e+00,   9.89830000e+04],
       [  9.27000000e+04,   2.00492000e+05],
       [  1.00000000e+00,   1.98870000e+04]]), [0, 1]], [array([[  9.89830000e+04,   1.18968980e+07],
       [  1.00000000e+00,   9.27000000e+04],
       [  1.00000000e+00,   8.60000000e+01]]), [1]], [array([[  9.89830000e+04,   6.00000600e+06],
       [  1.00000000e+00,   1.27500000e+03],
       [  8.60000000e+01,   1.98870000e+04]]), [1, 2]], [array([[  9.89830000e+04,   1.57949000e+06],
       [  1.27500000e+03,   4.66970000e+04],
       [  8.60000000e+01,   2.54900000e+03]]), [2, 3]], [array([[  9.89830000e+04,   1.57949000e+06],
       [  1.27500000e+03,   4.66970000e+04],
       [  2.54900000e+03,   5.02900000e+03]]), [3]], [array([[  9.89830000e+04,   1.57949000e+06],
       [  4.66970000e+04,   9.19720000e+04],
       [  8.60000000e+01,   2.53900

In [110]:
# === Performance Evaluation (dense) ===
#
# evaluate the blocks of data to be fetched, when physical data of kdnodes are dense !!!
# @queries: a collection of queries contains the lower and upper value in all dimensions; numpy object
# @kdnodes: the kdnodes generated above; array object
#
def Query(queries, dense_kdnodes):
    
    counts = []
    count_single_query = 0;
    
    # number of dimensions
    dims = int(len(queries[0])/2)
    
    # for each query
    for i in range(len(queries)):
        
        pages = []
        
        # check for intersection for each kdnode
        for j in range(len(dense_kdnodes)):
            
            # for each dimension
            intersection_tag = True
            for k in range(dims):
                
                # an intersection holds if it intersecs in all dimensions
                if queries[i][2*k] >= kdnodes[j][0][k][1] or queries[i][2*k+1] <= kdnodes[j][0][k][0]:
                    intersection_tag = False
                    break
                
            # if the query intersect with this kdnode
            if intersection_tag:
                pages.extend(dense_kdnodes[j][1]) # remember to remove repeated
        
        counts.append(len(set(pages)))
    
    print("blocks IO: ", counts)
    print("blocks IO(average): ", statistics.mean(counts))

In [111]:
# Example of usage (Performance Evaluation (dense))
#
Query(test_set, dense_kdnodes)

blocks IO:  [2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 4, 2, 2, 2, 2, 3, 2, 2, 2, 2, 1, 2, 4, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 4, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 3, 2, 3, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 3, 2, 3, 3, 1, 2, 2, 2, 2, 1, 2, 4, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 4, 2, 1, 2, 4, 1, 1, 3, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 4, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 